# Knowledge Ditillation

# Importing all the neccesary libraries

In [1]:
import numpy as np
import pandas as pd
import time
import os,sys
sys.path.append('../')
from sentence_transformers import SentenceTransformer
from transformers import AutoModel
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from src.model_evaluation import evaluation
from src.other_function import functions
from src.distillation_class import KnowledgeDistillation



/Users/msaqib/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/msaqib/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# Creating the object of evaluation class and other function class

In [2]:
eval_object=evaluation()

func_object=functions()

# loading the data

In [2]:
df=pd.read_csv("..//Data/processed_data.csv")


list_data = df['tokenized_docstring'].tolist()

# Loading the testing data

In [3]:
queries=pd.read_csv("..//Data/query.csv")

# Loading the teacher model and student model

In [4]:
teacher_model=SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

student_model=SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

# Architecture of the student Model


In [7]:
student_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

print(student_model)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

# Creating the object of knowlegde distillation class

In [5]:
know_dist_object=KnowledgeDistillation("all-mpnet-base-v2","all-MiniLM-L12-v2")


# Training the student model on the teacher model

In [1]:
know_dist_object.train_student_model()

# Loading the model from local drive

In [32]:
distillated_model=SentenceTransformer('..//models/distiiled_model')

# Architecture of distillated model

In [27]:
model_path = "..//models/distiiled_model"
distillated_model_arch = AutoModel.from_pretrained(model_path)

# Print the model architecture
print(distillated_model_arch)

# Our distillate model has 12 layers with the vector dimension size of 384 

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

# generating the embeddings of distillated model

In [35]:
func_object.get_model_embedding(distillated_model,"distilled_model")

starting_time : 1713431072.99
ending_time : 1713431400.05
Embedding_shape : (113884, 384)
total_time_taken for embedding generation:  327.06 seconds 


# size of distillated model

In [38]:
size=func_object.get_model_size("..//models/distilled_model_with_6_layer")

print(f"Size of 6 layer model is {(size/(1024*1024)):.2f} MB")

Size of 6 layer model is 148.20 MB


# size of embeddings

In [39]:
size_of_embeddings=func_object.get_embedding_size("..//embeddings/embeddings_distilled_model_with_6_layer.npy")

print(f"Size of embeddings of 6 layer model is {(size_of_embeddings/(1024*1024)):.2f} MB")

Size of embeddings of 6 layer model is 333.64 MB


# Response on the testing data

In [42]:
# order in which column of databse will be present
column_order = ['Query', 'Docstrings', 'Code','Match']

#creating the dataframe for model_1 which consists of for particular query all top code and
#match column shows whether it is correctly retrieved or not
dist_model_query_response=pd.DataFrame(columns=column_order)

distillated_model=SentenceTransformer('..//models/distiiled_model')

emb_dist_model=np.load("..//embeddings/embeddings_distilled_model.npy")

print(emb_dist_model.shape)

for i in range(len(queries)):
    #getting the particular query from queries 
    query=queries.loc[i].Questions
    
    # getting the table dataframe that consists of docstring and code 
    #that has top match with query using cosine similarity
    table=eval_object.get_top_code_and_docstring(query,emb_dist_model,distillated_model,list_data,df)
    
    #adding the column query with all values as 
    table=add_column(table,query)
    
    #adding columns Match that defines whether the code for given query is correctly retrieved or not
    table=add_column(table,'Match')
    
    #reordering the table for better view
    table=reorder_columns(table)
    
    #iterating in the table
    for index,row in table.iterrows():
        
        #fetching the current code for query
        result = row['Code']
        
        #getting the response from claude api
        response=eval_object.check_response(query,row['Code'])
    
        #Checking the value present in the response generated by the claude
        if 'YES' in response.content:
            response='YES'
        else :
            response='NO'
            
        #Giving the value of response in match column for current row
        table.at[index,'Match']=response
    
    #merging the table dataframe and model_1_query_response in single dataframe
    dist_model_query_response= pd.concat([dist_model_query_response, table], ignore_index=True)

(113884, 384)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# MAP@10 of distilled model

In [64]:
total_yes = (dist_model_query_response['Match'] == 'YES').sum()
total_no = (dist_model_query_response['Match'] == 'NO').sum()

accuracy_dist_model=(total_yes/(total_yes+total_no))*100

print(f"MAP@10 (mean average precision) of 8 layer model : {accuracy_dist_model}")


MAP@10 (mean average precision) of 8 layer model : 62.63157894736842
